In [1]:
import torch
import sys
import yaml
from torchvision import transforms, datasets
import torchvision
import numpy as np
import os
from sklearn import preprocessing
from torch.utils.data.dataloader import DataLoader

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#sys.path.append('../')
from model_utils.resnet_base_network import ResNet
from data_utils.transforms import get_data_transforms

In [5]:
batch_size = 32
data_transform = get_data_transforms(input_shape=(96, 96))

In [7]:
config = yaml.load(open("./configs/exp1.yaml", "r"), Loader=yaml.FullLoader)

In [53]:
"""
train_dataset = datasets.STL10('/home/thalles/Downloads/', split='train', download=False,
                               transform=data_transforms)

test_dataset = datasets.STL10('/home/thalles/Downloads/', split='test', download=False,
                               transform=data_transforms)
"""

"\ntrain_dataset = datasets.STL10('/home/thalles/Downloads/', split='train', download=False,\n                               transform=data_transforms)\n\ntest_dataset = datasets.STL10('/home/thalles/Downloads/', split='test', download=False,\n                               transform=data_transforms)\n"

In [9]:
from data_utils.usecase1 import UseCase1
from data_utils.balanced_split import balanced_split

data_path = "../data/happy_hoppy"

data_custom = UseCase1(targ_dir = data_path, transform = data_transform)

split = 0.3

train_dataset, test_dataset = balanced_split(data_custom, split)

In [10]:
print("Input shape:", train_dataset[0][0].shape)

Input shape: torch.Size([1, 64, 64])


In [11]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True)

In [12]:
device = 'cpu' #'cuda' if torch.cuda.is_available() else 'cpu'
encoder = ResNet(**config['network'])
output_feature_dim = encoder.projetion.net[0].in_features

In [16]:
import matplotlib.pyplot as plt

In [13]:
for batch_view_1, _ in train_loader:
    grid = torchvision.utils.make_grid(batch_view_1[:32])
    grid

In [17]:
plt.show(grid)

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [58]:
#load pre-trained parameters
load_params = torch.load(os.path.join('../runs/Jan12_14-36-20_1e89623a6adf/checkpoints/model.pth'),
                         map_location=torch.device(torch.device(device)))

if 'online_network_state_dict' in load_params:
    encoder.load_state_dict(load_params['online_network_state_dict'])
    print("Parameters successfully loaded.")

# remove the projection head
encoder = torch.nn.Sequential(*list(encoder.children())[:-1])
encoder = encoder.to(device)

Parameters successfully loaded.


In [59]:
encoder

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [60]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)

In [61]:
logreg = LogisticRegression(output_feature_dim, 10)
logreg = logreg.to(device)

In [62]:
def get_features_from_encoder(encoder, loader):
    
    x_train = []
    y_train = []

    # get the features from the pre-trained model
    for i, (x, y) in enumerate(loader):
        with torch.no_grad():
            feature_vector = encoder(x)
            x_train.extend(feature_vector)
            y_train.extend(y.numpy())

            
    x_train = torch.stack(x_train)
    y_train = torch.tensor(y_train)
    return x_train, y_train

In [63]:
encoder.eval()
x_train, y_train = get_features_from_encoder(encoder, train_loader)
x_test, y_test = get_features_from_encoder(encoder, test_loader)

if len(x_train.shape) > 2:
    x_train = torch.mean(x_train, dim=[2, 3])
    x_test = torch.mean(x_test, dim=[2, 3])
    
print("Training data shape:", x_train.shape, y_train.shape)
print("Testing data shape:", x_test.shape, y_test.shape)

Training data shape: torch.Size([4864, 512]) torch.Size([4864])
Testing data shape: torch.Size([11350, 512]) torch.Size([11350])


In [64]:
def create_data_loaders_from_arrays(X_train, y_train, X_test, y_test):

    train = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)

    test = torch.utils.data.TensorDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
    return train_loader, test_loader

In [65]:
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train).astype(np.float32)
x_test = scaler.transform(x_test).astype(np.float32)

In [66]:
train_loader, test_loader = create_data_loaders_from_arrays(torch.from_numpy(x_train), y_train, torch.from_numpy(x_test), y_test)

In [81]:
optimizer = torch.optim.Adam(logreg.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()
eval_every_n_epochs = 10

for epoch in range(20):
#     train_acc = []
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()        
        
        logits = logreg(x)
        predictions = torch.argmax(logits, dim=1)
        
        loss = criterion(logits, y)
        
        loss.backward()
        optimizer.step()
    
    total = 0
    if epoch % eval_every_n_epochs == 0:
        correct = 0
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            logits = logreg(x)
            predictions = torch.argmax(logits, dim=1)
            
            total += y.size(0)
            correct += (predictions == y).sum().item()
            
        acc = 100 * correct / total
        print(f"Testing accuracy: {np.mean(acc)}")

Testing accuracy: 76.77533039647577
Testing accuracy: 76.70484581497797


In [85]:
for x, y in test_loader:
    x = x.to(device)
    y = y.to(device)
    repr = encoder.forward(x)

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [512, 512]

TypeError: conv2d() received an invalid combination of arguments - got (DataLoader, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!DataLoader!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!DataLoader!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)
